# Complete PH workflow

The **PH** package has a very modular structure for dealing with the different parts of the **PH** benchmark test case:

* Inside the package **PH.ansatzes** all the modules related to ansatz circuit creation and solving are stored (see notebook: **02_Ansatzes.ipynb**)
* Inside the package **PH.parent_hamiltonian** all the modules related to the computation of the **PH** and its corresponding Pauli String decomposition are located (see notebook **03_Using_PH_Class.ipynb**)
* Inside the package **PH.ph_step_exe** the modules related to the computation of the ground state energy for a given ansatz and parent hamiltonian are stored (see notebook **04_ParentHamiltonian_execution.ipynb**).

In this notebook, the **PH.workflow.workflow** module is reviewed. This module groups all the aforementioned modules for executing a complete **PH** computation, this is:

1. Create the ansatz and compute its corresponding state.
2. Compute the corresponding **PH** and its Pauli decomposition.
3. Compute the ground state energy of the ansatz under its corresponding **PH**.

In [ ]:
import logging
logging.basicConfig(
    format='%(asctime)s-%(levelname)s: %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    level=logging.INFO
    #level=logging.DEBUG
)
logger = logging.getLogger('__name__')

In [ ]:
import numpy as np
import pandas as pd
import time

In [ ]:
import sys
sys.path.append("../../")

## 0. The QPU

To simulate the quantum circuits generated by the functions presented in this notebook a configured myQLM (or QLM) **Quantum Process Unit (QPU)** is mandatory. The **QPU** can execute an ideal simulation or can simulate the quantum circuits under a noisy hardware model (noisy simulation). To easily deal with these 2 kinds of simulations the  *select_qpu* function from **tnbs.qpu.select_qpu** module was developed. The input of this function is a Python dictionary that allows to the user configure easily a **QPU**.

In the present notebook, only ideal simulation is used. Please refer to the **PH/qpu/NoisyModels.ipynb** notebook for configuring noisy models and the corresponding noisy **QPU**s

In [ ]:
# myQLM qpus
sys.path.append("../../../")
from qpu.select_qpu import select_qpu

The minimum Python dictionary for configuring an ideal **QPU** is presented in the following cell. In this case, the user only has to provide a value to the *qpu_type* key. Depending on the type of simulator desired the following strings should be provided:

* *qlmass_linalg*: to use the **LinAlg Quantum Learning Machine (QLM)** algebra simulator. In this case, the computation will be sent to the **QLM** by using the  Qaptiva QLM as a Service.
* *qlmass_mps*: to use **MPS QLM** simulator. In this case, the computation will be sent to the **QLM** by using the  Qaptiva QLM as a Service.
* *python*: to use the PyLinalg algebra simulator.
* *c*: to use the CLinalg alegbra simulator.
* *linalg*: to use the **LinAlg QLM**. In this case, the user should be inside a **EVIDEN QLM**
* *mps*: to use the **MPS QLM** simulator. In this case, the user should be inside a **EVIDEN QLM**

In [ ]:
# List with the strings that should be provided for an ideal QPU
ideal_qpus = ["c", "python", "linalg", "mps", "qlmass_linalg", "qlmass_mps"]
qpu_config = {
    "qpu_type": ideal_qpus[0], 
}
qpu_ansatz = select_qpu(qpu_config)
qpu_ph = select_qpu(qpu_config)

In [ ]:
from PH.workflow.workflow import workflow

In order to use the *workflow* function from **PH.workflow.workflow** module a configuration dictionary should be provided. Following cell show a complete configuration

In [ ]:
nqubits = 8
depth = 4
i = 0
ansatz = ["simple01", "simple02", "lda", "hwe"][i]
if i== 0:
    t_inv = True
else:
    t_inv = False
truncation = None
nb_shots = 0
save = True
folder = "Savings_notebook5/"
configuration = {
    # ansatz related configuration
    "nqubits": nqubits,
    "depth": depth,
    "ansatz": ansatz,
    # parent hamiltonian configuration
    "t_inv": t_inv,
    # vqe step configuration
    "truncation": truncation,
    "nb_shots": nb_shots,
    "ansatz_qpu": qpu_ansatz,
    "ph_qpu": qpu_ph,
    "save": save,
    "folder": folder
    
}

In [ ]:
pdf = workflow(**configuration)

In [ ]:
pdf

We can truncate Pauli terms !!

In [ ]:
configuration.update({"truncation": 3})

In [ ]:
pdf = workflow(**configuration)

In [ ]:
pdf

In [ ]:
configuration.update({"truncation": 2})
pdf = workflow(**configuration)

In [ ]:
pdf

## Command Line

The **workflow** module can be executed from the command line. Several arguments can be provided for configuring the complete execution. To get a help type:

    python workflow.py -h

    usage: workflow.py [-h] [-nqubits NQUBITS] [-depth DEPTH] [-ansatz ANSATZ]
                       [-qpu_ansatz QPU_ANSATZ] [--t_inv] [-truncation TRUNCATION]
                       [-nb_shots NB_SHOTS] [-qpu_ph QPU_PH] [-qpu_id QPU_ID] [-folder FOLDER]
                       [--save] [--print] [--exe]

    optional arguments:
      -h, --help            show this help message and exit
      -nqubits NQUBITS      Number of qbits for the ansatz.
      -depth DEPTH          Depth for ansatz.
      -ansatz ANSATZ        Ansatz type: simple01, simple02, lda or hwe.
      -qpu_ansatz QPU_ANSATZ
                            QPU for parent hamiltonian simulation: c, python, linalg, mps,
                            qlmass_linalg, qlmass_mps
      --t_inv               Setting translational invariant of the ansatz
      -truncation TRUNCATION
                            Truncation for Pauli coeficients.
      -nb_shots NB_SHOTS    Number of shots
      -qpu_ph QPU_PH        JSON with the qpu configuration for ground state computation
      -qpu_id QPU_ID        Select a QPU for ground state computation from a JSON file
      -folder FOLDER        Path for storing results
      --save                For storing results
      --print               For printing the selected QPU configuration.
      --exe                 For executing program

**BE AWARE**

The configuration of the QPU for solving the ground state energy computation should be provided as JSON file (*-qpu_ph QPU_PH* argument). Example of these files are the:

* qpu_ideal.json
* qpu_noisy.json

that can be found in the **tnbs/qpu/** folder.

These JSON files can generate several QPUs configurations with the same file so we need to select which one will be used for simulation. This can be done by providing the *-qpu_id QPU_ID* argument (in combination with the *-json_qpu JSON_QPU* argument for providing the JSON file with the QPU configuration.


**Example**

If the following command is used:

    python workflow.py -ansatz simple02 -nqubits 6 -depth 3 -truncation 3 -nb_shots 0 -qpu_ansatz c -qpu_ph ../../../qpu/qpu_ideal.json -qpu_id 0 -folder Saves2 --print  --exe --save 
    
Then we are going to develop the complete workflow for an ansatz *simple_02* (*-ansatz simple02*) of 6 *qubits* (*-nqubits 6*) and with *depth* 3 (*-depth 3*). We are going to use a *truncation* of 3 ($10^{-3}$) (*-truncation 3*) for pruning the Pauli decomposition. For solving the ansatz the *CLinalg* qpu (*-qpu_ansatz c*) and for solving the ground state computation we use the configuration JSON file *../qpu/qpu_ideal.json* (*-qpu_ph ../qpu/qpu_ideal.json*) and we select the 0 element of the qpu list configurations (*-qpu_id 0*). We are going to save all into the *Saves2* folder.

In the *Saves2* folder the following files should be found:

* Saves2/ansatz_simple02_nqubits_6_depth_3_qpu_ansatz_CLinalg_parameters.csv
* Saves2/ansatz_simple02_nqubits_6_depth_3_qpu_ansatz_CLinalg_pauli.csv
* Saves2/ansatz_simple02_nqubits_6_depth_3_qpu_ansatz_CLinalg_ph_time.csv
* Saves2/ansatz_simple02_nqubits_6_depth_3_qpu_ansatz_CLinalg_phexe.csv
* Saves2/ansatz_simple02_nqubits_6_depth_3_qpu_ansatz_CLinalg_solve_ansatz_time.csv
* Saves2/ansatz_simple02_nqubits_6_depth_3_qpu_ansatz_CLinalg_state.csv
* Saves2/ansatz_simple02_nqubits_6_depth_3_qpu_ansatz_CLinalg_workflow.csv

## Masive Execution

As in the other cases we can execute masive executions of the **workflow** module. The mandatory files are:

* **workflow.json**: JSON file with the complete desired configuration for the workflow. Each posible configuration parameter in the JSON file has associated a list with the desired values. The total number of workflows will be all the posible combinations of the values of all the parameters. 
    * Example: if *nqubits: [10, 14]*, *depth: [1, 2, 3, 4]* and *ansatz: ["simple01", "simple02"]* (and the otheres parameters has only a one element) then the posible workflows will be 2 * 4 * 2= 16. 
* **launch_workflow.py** this script allows to configure the workflows taking the configuration from the **workflow.json** and executes them. 

Use **python launch_workflow -h** for getting a help. The following arguments can be provided:

    usage: launch_workflow.py [-h] [--all | -id ID] [-qpu_ph QPU_PH] [-qpu_id QPU_ID] [--print]
                              [--exe] [--count]

    options:
      -h, --help      show this help message and exit
      --all           For executing complete list
      -id ID          For executing only one element of the list
      -qpu_ph QPU_PH  JSON with the qpu configuration for ground state computation
      -qpu_id QPU_ID  Select which qpu to use from all possibilities
      --print         For printing the AE algorihtm configuration.
      --exe           For executing program
      --count         Getting the number of elements
    
**NOTE**

The user can provided a *JSON* file with several **QPU** configurations using the **-qpu_ph** and with the **-qpu_id** can specify which one should be used for executing all the workflows defined in the **workflow.json** file.

The **-id** argument controls which workflow of the all the possible ones shoud be executed (**--all** for executing all of them).

